# Task: Part of speech tagging

In this task we try to recreate a very rudimentary POS tagger "from scratch" using SpaCy and CRF models. 

(We disregard the fact, that SpaCy has a built in POS tagger for the moment for demonstration purposes.)

The input is a tokenized English sentence. The task is to label each word with a part of speech (POS) tag. The tag set, which is identical the [Universal Dependencies project's](https://universaldependencies.org/) basic tag set is the following:

- NOUN: noun
- VERB: verb
- DET: determiner
- ADJ: adjective
- ADP: adposition (e.g., prepositions)
- ADV: adverb
- CONJ: conjunction
- NUM: numeral
- PART: particle (function word that cannot be inflected, has no meaning in
  itself and doesn't fit elsewhere, e.g., "to")
- PRON: pronoun
- .: punctuation
- X: other

The code in this task is an adaptation of the NER code in the sklearn-crfsuite documentation.

# The data set

__Brown__ corpus: "The Brown University Standard Corpus of Present-Day American English (or just Brown Corpus) was compiled in the 1960s by Henry Kučera and W. Nelson Francis at Brown University, Providence, Rhode Island as a general corpus (text collection) in the field of corpus linguistics. It contains 500 samples of English-language text, totaling roughly one million words, compiled from works published in the United States in 1961" (Wikpedia: Brown Corpus)

Let's download and inspect the data!

In [1]:
%%capture
!pip install nltk

In [2]:
import nltk

from nltk.corpus import brown
nltk.download('brown')

brown.words()

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]

In [3]:
nltk.download('universal_tagset')
brown.tagged_words(tagset='universal')

[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


[('The', 'DET'), ('Fulton', 'NOUN'), ...]

In [4]:
brown.sents()

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ...]

In [5]:
len(brown.words())

1161192

From the brown the object provided by NLTK we will work with the tagged sentence list:

In [6]:
sents = brown.tagged_sents(tagset="universal")

sents[:2]

[[('The', 'DET'),
  ('Fulton', 'NOUN'),
  ('County', 'NOUN'),
  ('Grand', 'ADJ'),
  ('Jury', 'NOUN'),
  ('said', 'VERB'),
  ('Friday', 'NOUN'),
  ('an', 'DET'),
  ('investigation', 'NOUN'),
  ('of', 'ADP'),
  ("Atlanta's", 'NOUN'),
  ('recent', 'ADJ'),
  ('primary', 'NOUN'),
  ('election', 'NOUN'),
  ('produced', 'VERB'),
  ('``', '.'),
  ('no', 'DET'),
  ('evidence', 'NOUN'),
  ("''", '.'),
  ('that', 'ADP'),
  ('any', 'DET'),
  ('irregularities', 'NOUN'),
  ('took', 'VERB'),
  ('place', 'NOUN'),
  ('.', '.')],
 [('The', 'DET'),
  ('jury', 'NOUN'),
  ('further', 'ADV'),
  ('said', 'VERB'),
  ('in', 'ADP'),
  ('term-end', 'NOUN'),
  ('presentments', 'NOUN'),
  ('that', 'ADP'),
  ('the', 'DET'),
  ('City', 'NOUN'),
  ('Executive', 'ADJ'),
  ('Committee', 'NOUN'),
  (',', '.'),
  ('which', 'DET'),
  ('had', 'VERB'),
  ('over-all', 'ADJ'),
  ('charge', 'NOUN'),
  ('of', 'ADP'),
  ('the', 'DET'),
  ('election', 'NOUN'),
  (',', '.'),
  ('``', '.'),
  ('deserves', 'VERB'),
  ('the', 'DET'),

In [7]:
len(sents)

57340

We divide our data set into a train and a valid part:

In [8]:
valid_sents = sents[:5734]
train_sents = sents[5734:]

# Feature template

Since the plan is to build a CRF model, we need a __feature template__, which generates features for a word in a sentence (our sequence in the sequence tagging task). We use spaCy for feature extraction.

In [9]:
#!python -m spacy download en_core_web_sm
#Spacy install, load and such stuff

#Import
import spacy

#By model load, please deactivate unnecessary pipeline elements!
en = spacy.load("en_core_web_sm" , disable=["passer","ner"])


We write a function which generates features for a token in a sentence, which is already a spaCy document. The feature vector is represented as a `dict` mapping feature names to their values.

The desired **feature set for a token is**:

- `bias`: A constant value of 1 as an input
- `token.lower`: the lowercased textual form of the token
- `token.suffix`: the textual form of the token's suffix as defined by SpaCy,
- `token.prefix`: the textual form of the token's prefix as defined by SpaCy,
- `token.is_upper`: boolean value indicating if the token is uppercase,
- `token.is_title`: boolean value indicating if the token is a title,
- `token.is_digit`: boolean value indicating if the token consists of numbers.

These are only the `Token`'s own properties, but they represent no context.

We would like to include information about  the previous and next words, as well as indicating if the `Token` is the beginning or the end of sentence.

The **contextual features** should be:
 
- `-1:token.lower`: What is the lowercase textual form of the previous token?,
- `-1:token.is_title`: Is the previous token a title?,
- `-1:token.is_upper`: Is the previous token uppercase?,
- `+1:token.lower`: What is the lowercase textual form of the next token?,
- `+1:token.is_title`: Is the next token a title?,
- `+1:token.is_upper`: Is the next token uppercase?,
- `BOS`: Boolean value indicating if the token is the beginning of a sentence,
- `EOS`: Boolean value indicating if the token is the end of a sentence

In [60]:
def token2features(sent, i):
    """Return a feature dict for a token. 
    sent is a spaCy Doc containing a sentence, i is the token's index in it.
    """
    features = {}
    # For the word
    features['bias'] = 1.0
    features['token.lower']  = sent[i].lower_
    features['token.suffix'] = sent[i].suffix_
    features['token.prefix'] =  sent[i].prefix_
    features['token.is_upper'] = sent[i].is_upper
    features['token.is_title'] =  sent[i].is_title
    features['token.is_digit'] =  sent[i].is_digit

    # If token is previous
    if i > 0:
            features['-1:token.lower'] =  sent[i-1].lower_ 
            features['-1:token.is_title'] = sent[i-1].is_title
            features['-1:token.is_upper']=  sent[i-1].is_upper
    else:
        features['BOS'] =  True

    # If token is next
    if i < len(sent) - 1:
            features['+1:token.lower'] =  sent[i+1].lower_
            features['+1:token.is_title'] = sent[i+1].is_title
            features['+1:token.is_upper'] =  sent[i+1].is_upper
    else:
        features['EOS'] = True


    return features

For training, we will also need functions to generate feature dict and label lists for sentences in our training corpus:

In [61]:
from spacy.tokens import Doc
def sent2features(sent):
    "Return a list of feature dicts for a sentence in the data set."
    # Create a doc by instantiating a Doc class and iterating through the sentence token by token.
    # Please bear in mind, that Brown has token-POS pairs, latter one we don't need here...
    #....
    doc = [token[0] for token in sent]
    text = Doc(en.vocab, words=doc)
    
    # Plese use the above defined token2features function on each token to generate the features
    # For the whole sentence!
    sent_features = []
    for i in range(len(text)):
         sent_features.append(token2features(text, i))
    
    return sent_features

def sent2labels(sent):
    
    #Please create / filter only the labels for given sentence!
    labels = [token[1] for token in sent]
    
    return labels

Sanity check: let's see the values for the first 2 tokens in the corpus:

In [62]:
print(sent2features(sents[0])[:2])
print(sent2labels(sents[0])[:2])

[{'bias': 1.0, 'token.lower': 'the', 'token.suffix': 'The', 'token.prefix': 'T', 'token.is_upper': False, 'token.is_title': True, 'token.is_digit': False, 'BOS': True, '+1:token.lower': 'fulton', '+1:token.is_title': True, '+1:token.is_upper': False}, {'bias': 1.0, 'token.lower': 'fulton', 'token.suffix': 'ton', 'token.prefix': 'F', 'token.is_upper': False, 'token.is_title': True, 'token.is_digit': False, '-1:token.lower': 'the', '-1:token.is_title': True, '-1:token.is_upper': False, '+1:token.lower': 'county', '+1:token.is_title': True, '+1:token.is_upper': False}]
['DET', 'NOUN']


# Putting the data into final form

Everything is ready to generate the training data in the form which is usable for the CRFsuite. Note that our inputs and labels will be  2-level representations, lists of lists, because we deal with token sequences (sentences).

In [63]:
%%time
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_valid = [sent2features(s) for s in valid_sents]
y_valid = [sent2labels(s) for s in valid_sents]

CPU times: user 18.2 s, sys: 199 ms, total: 18.4 s
Wall time: 18.4 s


In [64]:
print("Feature dict for the first token in the first validation sentence:")
print(X_valid[0][0])
print("Its label:")
print(y_valid[0][0])

Feature dict for the first token in the first validation sentence:
{'bias': 1.0, 'token.lower': 'the', 'token.suffix': 'The', 'token.prefix': 'T', 'token.is_upper': False, 'token.is_title': True, 'token.is_digit': False, 'BOS': True, '+1:token.lower': 'fulton', '+1:token.is_title': True, '+1:token.is_upper': False}
Its label:
DET


# Training and evaluation

We use the super-optimized [CRFsuite](http://www.chokkan.org/software/crfsuite/) via the scikit-learn compatible [sklearn-crfsuite](https://sklearn-crfsuite.readthedocs.io) wrapper to train a CRF model on the data.

In [16]:
#%%capture # only to avoid ugly printouts during install
!pip install sklearn_crfsuite

     |████████████████████████████████| 747kB 18.2MB/s 


In [65]:
# Please import and train an averaged perceptron model from CRFsuite and use it's custom metrics,
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

# especially the multiple forms of accuracy score to evaluate the model!
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.2,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True)

In [67]:
%%time
crf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.2, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [69]:
# Please draw some conclusion if this model is "good enough" 
# in your view if you take token level and sentence level metrics into account!
labels = list(crf.classes_)
y_pred = crf.predict(X_valid)
metrics.flat_f1_score(y_valid, y_pred, 
                      average='weighted', labels=labels)

0.9764343387570985

- The model is one of the best with high accuracy score

- If we take token level features and sentence level metrics, it gives us more features to train our model which has more coorelation with target variable

Let's instantiate and fit our model. CRFsuite implements several learning methods, here we use "ap", i.e., averaged perceptron.

# Demonstration

Just for the fun, we can try out the model.

In [70]:
def predict_tags(sent):
    """Predict tags for a sentence.
    sent is a string.
    """
    doc = en(sent)
    return crf.predict([[token2features(doc, i) for i in range(len(doc))]])

In [72]:
 while True:
        sent = input("\nEnter a sentence to tag or press return to quit:\n")
        if sent:
            print(predict_tags(sent))
        else:
            print("\nEmpty input received -- bye!")
            break


Enter a sentence to tag or press return to quit:
This is an Englsh sentence.
[['DET', 'VERB', 'DET', 'ADJ', 'NOUN', '.']]

Enter a sentence to tag or press return to quit:


Empty input received -- bye!
